In [14]:
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from csv import reader
nltk.download('wordnet')
nltk.download('stopwords')
from collections import Counter
import re

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\william.brach\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\william.brach\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("RecipeExtracted.csv", sep="\t")
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()

In [3]:
df.head()

,Unnamed: 0,title,url,path_to_file,recipe_title,recipe_servings,recipe_ingredients,recipe_steps
0,0,Vegetable dhansak recipe - BBC Food,https://www.bbc.co.uk/food/recipes/vegetable_d...,./DATA//Vegetable dhansak recipe - BBC Food.html,Vegetable dhansak,4.0,"['1 small cauliflower, cut into small florets'...","['Preheat the oven to 200C/180C Fan/Gas 6.', '..."
1,1,Mary Berry's fish pie recipe - BBC Food,https://www.bbc.co.uk/food/recipes/mary_berrys...,./DATA//Mary Berry's fish pie recipe - BBC Foo...,Mary Berry&#x27;s fish pie,6.0,"['75g/3oz butter, plus extra for greasing', '1...",['Preheat the oven 200C/400F/Gas 6. Grease a ...
2,2,Vegan mushroom stroganoff recipe - BBC Food,https://www.bbc.co.uk/food/recipes/vegan_mushr...,./DATA//Vegan mushroom stroganoff recipe - BB...,Vegan mushroom stroganoff,2.0,['160g/5¾oz dried pappardelle or fettuccine pa...,['Cook the pasta in a large saucepan of salted...
3,3,Speedy sweet and sour pork recipe - BBC Food,https://www.bbc.co.uk/food/recipes/speedy_swee...,./DATA//Speedy sweet and sour pork recipe - BB...,Speedy sweet and sour pork,6.0,"['450g/1lb white rice', '600g/1lb 5oz pork min...","['Put the rice in a large saucepan, cover with..."
4,4,Cherry scones recipe - BBC Food,https://www.bbc.co.uk/food/recipes/scones_35420,./DATA//Cherry scones recipe - BBC Food.html,Cherry scones,8.0,"['225g/8oz self-raising flour, plus extra for ...",['Preheat the oven to 200C/Fan 180C/Gas 6. Lin...


In [4]:
def create_index(row,index_dict):
    s = row['recipe_ingredients']
    s = s[1:-1]

    pattern = r'\'|\"|,'
    s = re.sub(pattern, '', s)
    title = row['recipe_title'].strip()
    pattern2 = r'\s'
    s = re.sub(pattern2, r'\t',s)
    s = s.split("\t")

    ingred = list()
    for word in s:    
        if word in stopwords.words('english') or bool(re.search(r'\d', word)):
            continue
        stem = porter_stemmer.stem(word)
        if stem in index_dict:
            index_dict[stem].append(title)
        else:
            index_dict[stem] = list()
            index_dict[stem].append(title)
    return row

In [26]:
%%time

index_dict = dict()

# open file in read mode
with open('RecipeExtracted.csv', 'r',encoding='utf-8') as read_obj:
    csv_reader = reader(read_obj, delimiter="\t")
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            title = row[4].strip()
            s = row[6]
            s = s[1:-1]
            pattern = r'\'|\"|,'
            s = re.sub(pattern, '', s)
            pattern2 = r'\s'
            s = re.sub(pattern2, r'\t',s)
            s = s.split("\t")

            ingred = list()
            for word in s:    
                if word in stopwords.words('english') or bool(re.search(r'\d', word)):
                    continue
                stem = porter_stemmer.stem(word)
                if stem in index_dict:
                    index_dict[stem].append(title)
                else:
                    index_dict[stem] = list()
                    index_dict[stem].append(title)

['', 'title', 'url', 'path_to_file', 'recipe_title', 'recipe_servings', 'recipe_ingredients', 'recipe_steps']
Wall time: 1min 4s


In [5]:
# %%time
# index_dict = dict()
# df = df.apply(lambda row: create_index(row,index_dict), axis=1)

Wall time: 1min 4s


In [27]:
def searchRecipesByInput(x):
    input_arr = x.split(" ")
    resultList = list()
    for word in input_arr:
        stem = porter_stemmer.stem(word)        
        if stem in index_dict:
            resultList.extend(index_dict[stem])
    result = Counter(resultList)
    result = result.most_common(10)
    return result

In [34]:
print('Enter food:')
x = input()
recipes = searchRecipesByInput(x)
for index,recipe in enumerate(recipes):
    print("{}.Recipe- {}, Score - {}".format(index+1,recipe[0],recipe[1]))

Enter food:
lamb with cheese
1.Recipe- Slow roast shoulder of lamb with sliced potatoes, Score - 4
2.Recipe- Goats&#x27; cheese double-baked soufflés with walnut salad, Score - 4
3.Recipe- Lamb stew, Score - 3
4.Recipe- Midweek tacos using batch-cooked mince, Score - 3
5.Recipe- Chargilled lamb fillet with garlic puree, borlotti beans and wild garlic, Score - 3
6.Recipe- Slow-roasted lamb shoulder and homemade mint sauce, Score - 3
7.Recipe- Lancashire hotpot with black pudding, Score - 3
8.Recipe- Roast saddle of lamb with raspberry sauce, Score - 3
9.Recipe- Lamb shoulder with harissa and lamb stovies, Score - 3
10.Recipe- Lamb kofta curry, Score - 3
